# Менеджеры контекста

Менеджеры контекста это удобный способ управления ресурсами, например, файлами, сокетами и т.д. Это короткое представление паттерна, использующего конструкцию ```try ... finaly ...```:

```python
r = acquire_resource()  # получение ресурса
try:
    do_something(r)  # какие-либо операции с ресурсом
finaly:
    # освобождение происходит в любом случае
    release_resource(r)
```

Менеджер контекста позволяет оставить за кадром освобождение ресурсов, т.е. действие ```release_resource``` будет вызвано неявно:

```python
with acquire_resource as r:
    do_something(r)
```

Для поддержки оператора ```with``` необходимо, чтобы объект реализовывал протокол менеджера контекста. Этот протокол заключается в реализации двух методов: ```__enter__``` и ```__exit__```. Первый метод служит для инициализации контекста, например, открывает файл. Значение, возвращаемое ```__enter__```, записывается в переменную по имени, указанному после оператора ```as```. Второй метод вызывается после выполнения тела оператора ```with``` и принимает три аргумента: тип исключения, само исключение, объект типа ```traceback```. Если в процессе выполнения тела оператора ```with``` было поднято исключение, метод ```__exit__``` может подави его, вернув ```True```. Любые объекты, реализующие эти два метода, будут являться менеджерами контекста. Подобное поведение без явного указания реализуемого интерфейса называется утиной типизацией.

In [1]:
path = r'python_pd\05_files\\'
with open(path + 'data.txt', 'r', encoding='cp1251') as f:
    data = f.read()
print(f'{data = }')

data = 'foo\nbar\nbaz\nquz'


Оператор ```with``` позволяет работать одновременно с несколькими менеджерами контекста:

```python
with acquire_res() as r, acquire_other_res() as other:
    do_something(r, other)
```

Такая запись эквивалентна двум вложенным менеджерам контекста:

```python
with acquire_res() as r:
    with acquire_other_res() as other:
        do_something(r, other)
```

Однако не стоит злоупотреблять использованием первого варианта, особенно для большого количества менеджеров контекста. Порой читаемость кода будет лучше, если использовать вложенные блоки.

In [2]:
with open(path + 'data.txt') as f1, open(path + 'write_data.txt') as f2:
    data1 = f1.read()
    data2 = f2.read()
print(f'{data1 = }')
print(f'{data2 = }')

data1 = 'foo\nbar\nbaz\nquz'
data2 = 'Foo\tbar\n'


Оператор ```with``` можно использовать без явного указания имени переменной.

```python
with acquire_res():
    do_something()
```

Стандартная библиотека Python включает самые разные менеджеры контекста для различных ситуаций. При этом работа с ними единообразна. Например, модуль ```tempfile``` реализует классы для работы с временными файлами. Он содержит класс ```TemporaryFile```, который автоматически удаляет файлы при выходе из менеджера контекста. Также менеджеры контекста активно используются при асинхронной работе.

О том как реализовать собственный менеджер контекста читайте в главе про классы.

# Модуль ```contextlib```

Модуль ```contextlib``` входит в стандартную библиотеку и включает в себя такие менеджеры контекста как:

- ```closing``` – обобщение логики освобождения ресурсов, подходит для любых объектов реализующих метод ```close```;
- ```redirect_stdout``` – перенаправляет вывод в стандартный поток);
- ```suppress``` – позволяет подавить исключения указанных типов;
- ```ContextDecorator``` – объединяет менеджеры контекста и декораторы;
- ```ExitStack``` – позволяет управлять произвольным количеством менеджеров контекста (например, когда необходимо управлять произвольным количество ресурсов). 

Например, функция, логирующая пути открытых файлов:

In [3]:
from contextlib import ExitStack

def marge_log(path, *logs):
    """Логирование открытых файлов.
    Логирует имена отурытых файлов с помощью 
    класса contextlib.ExitStack.

    :param path: путь до лог-файла
    :type path: str
    :param logs: пути до файлов
    :type logs: tuple[str]
    """
    with ExitStack() as st:
        # добавляем произвольное количество ресурсов в стек
        handles = [st.enter_context(open(log)) for log in logs]
        # не забываем про файл логов
        output = open(path, 'w')
        st.enter_context(output)

        # пишем в файл логов имена всех остальных файлов
        output.write('\n'.join(h.name for h in handles))

Или использование ```suppress``` для элегантного удаления файла (пример из документации).

In [4]:
from contextlib import suppress

with suppress(FileNotFoundError):
    os.remove('somefile.tmp')

Пример применения декоратора ```contextmanager```, который может сделать менеджером контекста даже функцию. 

В качестве примера приведена функция ```get_prefix_spaces```, которая умеет выводить на экран сообщения с отступом. При этом учитываются вложенные менеджеры контекста.

In [5]:
from contextlib import contextmanager


def get_prefix_spaces(indent=4):
    """Фабрика менеджеров контекста печати с отступом.
    Для создания менеджера контекста из функции 
    используется декоратор contextlib.contextmanager.

    В качестве "хака" для отслеживания уровня отступа 
    используется объемлющая область видимости и оператор 
    nonlocal для возможности редактирования.

    Функция _print - это обертка для print.

    :param indent: количество пробелов в одном уровне отступа.
    :type indent: int
    :return: менеджер контекста для печати с отступом.
    :rtype: Callable
    """
    level = 0
    def _print(msg, *args, **kwargs):
        print(f'{" " * indent * level}{msg}', *args, **kwargs)

    @contextmanager
    def inner():
        nonlocal level
        try:
            level += 1
            yield _print
        finally:
            level -= 1
    return inner


prefix_spaces = get_prefix_spaces()

with prefix_spaces() as f:
    f('foo')
    with prefix_spaces() as f:
        f('bar')
        with prefix_spaces() as f:
            f('baz')


    foo
        bar
            baz


Читайте документацию по ```contextlib``` для ознакомления с другими вариантами менеджеров контекста и примерами их использования.

# Полезные ссылки

- [Документация по ```contextlib```](https://docs.python.org/3/library/contextlib.html)